## Computing the Joint Spectral Radius <br>
Date: 7th Jan 2023 <br>
Reference: Stability Analysis of Switched Linear Systems with Neural
Lyapunov Functions

We wish to compute the ellipsoidal approximation of the Joint Spectral Radius, defined as $\rho_{Q}(\Sigma) := \inf_{P > 0}{\max_{A \in \Sigma}{||A||_{P}}}$ <br>
In the above, $||A||_{P}$ denotes the matrix norm induced by the ellipsoidal norm associated to $P$, i.e. $||x||_{P} := \sqrt{x^TPx}$.   

For computing the ENAJSR, we refer to: [Blondel et al. "On the accuracy of the ellipsoid norm approximation of the joint spectral radius"](https://perso.uclouvain.be/vincent.blondel/publications/04BNT2-jsr.pdf)

An important friend would be John's Ellipsoid Theorem, which we state here. <br>
**John's Ellipsoid Theorem** : Let $K \in \mathbb{R}^n$ be a compact convex subset with nonempty interior. Then there is an ellipsoid $E$ with center $c$ such that the inclusions $E ⊆ K ⊆ n(E - c)$ hold. If $K$ is symmetric about the origin, i.e. $K = -K$, then constant $n$ can be changed to $\sqrt{n}$.

Further, we also might need methods to generate Positive Definite Matrices. We list down all methods we know here <br>
1. [Sklearn make_spd_matrix()](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_spd_matrix.html) <br>
2. [Stackoverflow thread on the topic](https://stackoverflow.com/questions/619335/a-simple-algorithm-for-generating-positive-semidefinite-matrices) <br>
3. [Wishart Distributions](https://en.wikipedia.org/wiki/Wishart_distribution) (Not exactly sure of what these are, but they look interesting).

###Identifying Positive Definite Matrices <br>
1. A matrix is positive definite if it's symmetric and all its eigenvalues are positive <br>
2. A matrix is positive definite if it's symmetric and all its pivots are positive <br>
3. A matrix is positive definite if $x^TAx > 0$ for all vectors $x \neq 0$ <br>
4. A matrix $A$ is positive definite iff it can be written as $A = R^TR$ for some possibly rectangular matrix $R$ with independent columns <br>


**Function 1 (Generate a positive definite matrix)** <br>
Function: Generate a positive definite matrix. Then compute all the eigenvalues of the matrix and print them.

In [ ]:
import numpy as np

def generate_positive_definite_matrix_with_parametrization(params):
    # Create a lower triangular matrix with the given parameters
    lower_triangular = np.array([[params[0], 0, 0],
                                 [params[1], params[2], 0],
                                 [params[3], params[4], params[5]]])

    # Construct the positive definite matrix using Cholesky decomposition
    matrix = np.dot(lower_triangular, lower_triangular.T)

    # Compute the eigenvalues of the matrix
    eigenvalues = np.linalg.eigvals(matrix)

    return matrix, eigenvalues


In [ ]:
# Example usage with a 6-parameter vector
parameters = np.array([-1, -2, -3, -4, -5, -6])
matrix1, eigenvalues1 = generate_positive_definite_matrix_with_parametrization(parameters)
print("Matrix: ", matrix1 )
print("Eigenvalues: ", eigenvalues1)

Matrix:  [[ 1  2  4]
 [ 2 13 23]
 [ 4 23 77]]
Eigenvalues:  [84.64176666  0.67334625  5.68488709]


**Function 2 (Cholesky Decomposition)** <br>
Given a positive definite matrix, factorize it using the Cholesky decomposition. <br>

In [ ]:
import numpy as np

def cholesky_decomposition(matrix):
    """
    Perform Cholesky decomposition on a positive definite matrix.

    Parameters:
    - matrix: Positive definite matrix to be factorized.

    Returns:
    - L: Lower triangular matrix such that matrix = LL^T.
    """
    try:
        # Perform Cholesky decomposition
        L = np.linalg.cholesky(matrix)
        return L
    except np.linalg.LinAlgError:
        # Handle the case where the input matrix is not positive definite
        raise ValueError("Input matrix is not positive definite. Cholesky decomposition cannot be performed.")

# Example usage with a positive definite matrix


In [ ]:
cholesky_factor = cholesky_decomposition(matrix1)
print("\nCholesky Factor (Lower Triangular Matrix):")
print(cholesky_factor)



Cholesky Factor (Lower Triangular Matrix):
[[1. 0. 0.]
 [2. 3. 0.]
 [4. 5. 6.]]


**Function 3 (Null Space)** <br>
Computes the null space of a given matrix in time complexity $\mathcal{O}(m,n,\min{(m,n)})$

In [ ]:
import numpy as np

def null_space(matrix):
    _, _, V = np.linalg.svd(matrix)
    null_space_basis = V.T[:, np.isclose(np.linalg.svd(matrix)[1], 0)]
    return null_space_basis

In [ ]:
# Example usage
A = np.array([[1, 2, 3],
              [4, 5, 6],
              [7, 8, 9]])

null_space_basis = null_space(A)
print("Null Space Basis:")
print(null_space_basis)

Null Space Basis:
[[-0.40824829]
 [ 0.81649658]
 [-0.40824829]]


In [ ]:
import numpy as np
from scipy.linalg import fractional_matrix_power

def joint_spectral_radius(matrix_set):
    max_norms = []
    for matrix in matrix_set:
        max_norms.append(np.max(np.abs(np.linalg.eigvals(matrix))))

    joint_radius = np.max(max_norms)
    return joint_radius

# Example usage with two matrices
matrix1 = np.array([[0.8, 0.6], [-0.6, 0.8]])
matrix2 = np.array([[0.5, -0.3], [0.3, 0.5]])

matrix_set = [matrix1, matrix2]

result = joint_spectral_radius(matrix_set)
print("Joint Spectral Radius:", result)
